This file trains and tests the LSTM classifier for the eeg data augmented with the mahalanobis distances of the samples from the event cluster centers.  The modes are sub_event and event.  Sub_event looks at the centers for each hand motion as being zero or one, while the event mode looks at each combination of hand motions

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Input, Flatten
from keras.layers.recurrent import LSTM
from keras.models import load_model, Model
import keras
from eeg_generator_augmented import DataGenerator

/Users/joenunez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 50
sample_dim = 32
steps_back = 50
target_dim = (6)
num_epochs = 3
n_evals=16
mode='sub_event'

train_generator = DataGenerator(batch_size=batch_size, 
    dim1=sample_dim, target_dim=target_dim, 
    steps_back=steps_back, mode=mode, n_evals=n_evals)

input_shape = train_generator.get_input_shape()


list_IDs filled


In [5]:
model = Sequential()
model.add(LSTM(1000, input_shape=input_shape))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(target_dim, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='nadam')
print('model compiled')

model compiled


In [6]:
model.fit_generator(generator=train_generator, epochs=num_epochs, 
                    steps_per_epoch=10000, workers=4, use_multiprocessing=True)
model.save("eeg_lstm_sub_aug_50_steps.h5")

Epoch 1/3
 67/100 [===================>..........] - ETA: 38s - loss: nan

Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/joenunez/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/joenunez/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/joenunez/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/joenunez/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/joenunez/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/joenunez/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/joenunez/anaconda3/lib/python3.6/multip

KeyboardInterrupt: 

In [3]:
xlist=[]
ylist=[]
for i in range(1000):
    x,y = train_generator.__getitem__(100000+i)
    xlist.append(x)
    ylist.append(y)

X_test = np.concatenate(xlist)
y_test = np.concatenate(ylist)
print('test samples made')

test samples made


In [4]:
model = load_model("eeg_lstm_sub_aug_50_steps.h5")
model.summary()
preds = model.predict(X_test)
print('preds made')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1000)              4180000   
_________________________________________________________________
dense_36 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_37 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_11 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_38 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_39 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_12 (Dropout)         (None, 1000)              0         
__________

Computes the accuracy.  Also possible to compile the model with accuracy as a metric and use evaluate function, but this lets us leverage the round function more easily than putting a specialized layer at the end of the network 

In [6]:
corr=float(0)
for i,p in enumerate(preds):
    if np.array_equal(np.around(p), y_test[i]):
        corr+=1
print(corr/len(preds))

0.82898
